In [1]:
import pandas as pd
import numpy as np
import re
from keras.models import Model
from keras.layers.core import Dropout, Dense, Flatten, Activation
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.layers.recurrent import GRU
from keras.layers.pooling import MaxPooling2D
from keras.layers import Merge,Input


Using TensorFlow backend.


In [2]:
df = pd.read_csv('quora_duplicate_questions.tsv',sep='\t',encoding='utf8')

In [3]:
df = df.dropna()

In [4]:
import spacy
#from gensim.utils import lemmatize
nlp = spacy.load('en')

In [5]:
def clean_data(string):
    #try: 
    rx = re.compile('\W+')
    res = rx.sub(' ',string).strip()
    res = res.replace('\'','')
    #return ' '.join([x for x in nlp(res)])
    
    return ' '.join([x.lemma_ for x in nlp(res) if not x.is_stop ])

    #except TypeError:
        #print('TypeError',string)

In [6]:
df['q1_clean'] = df['question1'].apply(lambda x: clean_data(x))
df['q2_clean'] = df['question2'].apply(lambda x: clean_data(x))

TypeError: sequence item 0: expected str instance, spacy.tokens.token.Token found

In [7]:
df.head()

,id,qid1,qid2,question1,question2,is_duplicate,q1_clean,q2_clean
0,0,1,2,What is the step by step guide to invest in sh...,What is the step by step guide to invest in sh...,0,step step guide invest share market india,step step guide invest share market
1,1,3,4,What is the story of Kohinoor (Koh-i-Noor) Dia...,What would happen if the Indian government sto...,0,story kohinoor koh noor diamond,happen indian government steal kohinoor koh no...
2,2,5,6,How can I increase the speed of my internet co...,How can Internet speed be increased by hacking...,0,increase speed internet connection vpn,internet speed increase hack dns
3,3,7,8,Why am I mentally very lonely? How can I solve...,Find the remainder when [math]23^{24}[/math] i...,0,mentally lonely solve,find remainder math 23 24 math divide 24 23
4,4,9,10,"Which one dissolve in water quikly sugar, salt...",Which fish would survive in salt water?,0,dissolve water quikly sugar salt methane carbo...,fish survive salt water


In [8]:
print('q1_counts \n',df['q1_clean'].apply(lambda x: len(x.split())).describe())
print('q2_counts \n',df['q2_clean'].apply(lambda x: len(x.split())).describe())

q1_counts 
 count    404288.000000
mean          5.222468
std           2.913420
min           0.000000
25%           3.000000
50%           5.000000
75%           6.000000
max          68.000000
Name: q1_clean, dtype: float64
q2_counts 
 count    404288.000000
mean          5.278838
std           3.182402
min           0.000000
25%           3.000000
50%           4.000000
75%           6.000000
max          92.000000
Name: q2_clean, dtype: float64


In [ ]:
word_vecs =np.array([word.vector for word in doc for doc in nlp.pipe(df['question1'], n_threads=50)])

In [9]:
def build_vocab(string):
    for x in string.split():
        vocab.add(x)
        lexicon.append(x)
    return

In [10]:
vocab = set()
lexicon = []

In [11]:
_ = df['q1_clean'].apply(lambda x: build_vocab(x))
_ = df['q2_clean'].apply(lambda x: build_vocab(x))

In [12]:
vocab_size = len(vocab)

In [13]:
vocab_size

77092

In [14]:
def generate_labels(inputs, cnter,max_len,vocab_len):
    np_input = []
    
    for sentences in inputs:
        sen  = np.full(max_len,vocab_len)
        for idx,word in enumerate(sentences.split()):
            if(idx<max_len):
                sen[idx] = cnter.index(word)
        np_input.append(sen)
        
    return np.array(np_input).astype(np.int)
        
        

In [15]:
from collections import Counter
vocab_cnter = Counter(lexicon)

In [16]:
sorted_vocab = [x[0] for x in vocab_cnter.most_common()]

In [17]:
sorted_vocab[:10]

['good', 's', 'india', 'like', 'people', 'way', 't', 'quora', 'learn', 'life']

In [18]:
word2vec = []
for word in sorted_vocab:
    word2vec.append(nlp(word).vector)

In [19]:
len(word2vec)

77092

In [20]:
word2vec = np.array(word2vec)
word2vec.shape

(77092, 300)

In [21]:
q1_inputs = generate_labels(df['q1_clean'],sorted_vocab,30,len(vocab))
print('q1 processed')
q2_inputs = generate_labels(df['q2_clean'],sorted_vocab,30,len(vocab))

q1 processed


In [22]:
from keras.models import Model
from keras.layers.core import  Lambda,Dropout,Dense, Flatten, Activation
from keras.layers.embeddings import Embedding
from keras.layers.wrappers import Bidirectional
from keras.layers.convolutional import Conv1D
from keras.layers.recurrent import GRU,LSTM
from keras.layers.pooling import MaxPooling2D
from keras.layers import Concatenate, Input, concatenate, BatchNormalization
from keras import backend as K
def cosine_distance(vests):
    x, y = vests
    x = K.l2_normalize(x, axis=-1)
    y = K.l2_normalize(y, axis=-1)
    return -K.mean(x * y, axis=-1, keepdims=True)
def euclidean_distance(vects):
    x, y = vects
    return K.sqrt(K.sum(K.square(x - y), axis=1, keepdims=True))

def eucl_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0], 1)

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))

def cos_dist_output_shape(shapes):
    shape1, shape2 = shapes
    return (shape1[0],1)

# Q1
input_1 = Input(shape=(30,))
embedd = Embedding(len(sorted_vocab),300,
                        weights =[word2vec],
                        name="Embedding_1")
embedding_1 = (embedd)(input_1)

bi_gru_1 = Bidirectional(LSTM(200,name='LSTM_1',dropout=.8,recurrent_dropout=.8,return_sequences=True))(embedding_1)
#bi_gru_1_2 =  Bidirectional(LSTM(100,name="LSTM_1_2",dropout=.8,recurrent_dropout=.8))(bi_gru_1)

#Q2 
input_2 = Input(shape=(30,))
embedding_2 = (embedd)(input_2)
bi_gru_2 = Bidirectional(LSTM(200,name="LSTM_2",dropout=.8,return_sequences=True))(embedding_2)
#bi_gru_2_2 =  Bidirectional(LSTM(100,name="LSTM_2_2",recurrent_dropout=.8))(bi_gru_2)

#Merge
#cosinedistance
merged_1 = Lambda(cosine_distance,output_shape=cos_dist_output_shape)([bi_gru_1,bi_gru_2])
#euclid distance
merged_2 = Lambda(euclidean_distance,output_shape=eucl_dist_output_shape)([bi_gru_1,bi_gru_2])
merged = concatenate([merged_1,merged_2])
#merged = (Flatten())(merged)
#Dense
dense = Dense(4098,activation='relu',name='dense')(merged)
drop_1 = Dropout(.5,name='drop_1')(dense)
dense_2 = Dense(1000,activation='relu',name='dense_2')(drop_1)
drop_2 = Dropout(.5,name='drop_2')(dense_2)
pred = Dense(2,activation="softmax",name='pred')(drop_2)
model = Model(inputs=[input_1,input_2],outputs=pred)
# model = Sequential()
# model.add(Embedding(len(vocab),64,input_length=sentence_len,name='Embedding_Input'))
# model.add(Bidirectional(GRU(100,dropout=.5,name='GRU')))
# model.add(Dense(1000,activation='relu',))

In [23]:
q1_inputs.shape

(404288, 30)

In [24]:
q2_inputs.shape

(404288, 30)

In [27]:

def contrastive_loss(y_true, y_pred):
    '''Contrastive loss from Hadsell-et-al.'06
    http://yann.lecun.com/exdb/publis/pdf/hadsell-chopra-lecun-06.pdf
    '''
    margin = 1
    return K.mean(y_true * K.square(y_pred) + (1 - y_true) * K.square(K.maximum(margin - y_pred, 0)))


In [24]:
from keras import optimizers
import keras
adam = optimizers.Adam(lr=.001)
tbCallBack = keras.callbacks.TensorBoard(log_dir='./Graph_2_lstm', histogram_freq=0, write_graph=True, write_images=True)
model.compile(loss='categorical_crossentropy',optimizer=adam,metrics=['accuracy'])
model.fit(x=[q1_inputs,q2_inputs],y=pd.get_dummies(df['is_duplicate']).values,
          batch_size=64,epochs=30,validation_split=.1,callbacks=[tbCallBack])

Train on 363859 samples, validate on 40429 samples
Epoch 1/30


InvalidArgumentError: ConcatOp : Dimensions of inputs should match: shape[0] = [64,30,1] vs. shape[1] = [64,1,400]
	 [[Node: concatenate_1/concat = ConcatV2[N=2, T=DT_FLOAT, Tidx=DT_INT32, _device="/job:localhost/replica:0/task:0/gpu:0"](lambda_1/Neg, lambda_2/Sqrt, concatenate_1/concat/axis)]]
	 [[Node: mul_101/_161 = _Recv[client_terminated=false, recv_device="/job:localhost/replica:0/task:0/cpu:0", send_device="/job:localhost/replica:0/task:0/gpu:0", send_device_incarnation=1, tensor_name="edge_10801_mul_101", tensor_type=DT_FLOAT, _device="/job:localhost/replica:0/task:0/cpu:0"]()]]

Caused by op 'concatenate_1/concat', defined at:
  File "/home/kevin/anaconda3/lib/python3.5/runpy.py", line 184, in _run_module_as_main
    "__main__", mod_spec)
  File "/home/kevin/anaconda3/lib/python3.5/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/home/kevin/anaconda3/lib/python3.5/site-packages/ipykernel/__main__.py", line 3, in <module>
    app.launch_new_instance()
  Fil